In [1]:
import pandas as pd
from sportsreference.ncaab.teams import Teams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt
import urllib.request


from datetime import datetime
import requests
import os
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('NBA_2016_thru_2022_seasons_w_playoffs.csv')
df = df[~df['Unnamed: 4'].isna()].reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date'], ascending=True, inplace=True)
df.drop(columns = ['Unnamed: 4'], inplace=True)
df.rename(columns={'Unnamed: 5':'home_away',
                  'Unnamed: 7': 'result',
                  'Unnamed: 8':'ot'}, inplace=True)


team_map = pd.read_csv('team_name_mapping.csv')

df = df.merge(team_map, left_on='Opponent', right_on='full_name', how='inner')
df.drop(columns=['Opponent', 'full_name'], inplace=True)

df.rename(columns= {'abbr':'opponent'}, inplace=True)


# df = df[df['home_away'].isna()].reset_index(drop=True)


# df.drop(columns=['home_away'], inplace=True)

# df['game_key'] = df['home_team'].astype(str) + '_' + df['G'].astype(str) + '_' + df['season'].astype(str)
df['game_key'] = ''

dfat = pd.DataFrame(df[df['home_away'] == '@'])
dfh = pd.DataFrame(df[df['home_away'].isna()])

dfat['game_key'] = dfat['teamgame'] + '_at_'+ dfat['opponent'] + '_' + dfat['Date'].astype(str)

dfh['game_key'] = dfh['opponent'] + '_at_'+ dfh['teamgame'] + '_' + dfh['Date'].astype(str)

df = pd.concat([dfat, dfh]).sort_values(by=['Date'])
df.reset_index(drop=True, inplace=True)

In [3]:
df_reg = pd.DataFrame(df[df['playoffs'] == 0])
df_pl = pd.DataFrame(df[df['playoffs'] == 1])

df_ppg = pd.DataFrame(df_reg.groupby(['teamgame', 'season'])['Tm'].mean()).reset_index()

df_ppg.rename(columns={'Tm': 'ppg'}, inplace=True)

df_pl = df_pl.merge(df_ppg, on=['teamgame', 'season'], how='inner')


In [4]:
df_pl

,G,Date,Start (ET),home_away,result,ot,Tm,Opp,W,L,Streak,playoffs,teamgame,season,opponent,game_key,ppg
0,1,2016-04-16,3:30p,NaN,W,NaN,104.0,78.0,1.0,0.0,W 1,1,GSW,2016,HOU,HOU_at_GSW_2016-04-16,114.890244
1,2,2016-04-18,10:30p,NaN,W,NaN,115.0,106.0,2.0,0.0,W 2,1,GSW,2016,HOU,HOU_at_GSW_2016-04-18,114.890244
2,3,2016-04-21,9:30p,@,L,NaN,96.0,97.0,2.0,1.0,L 1,1,GSW,2016,HOU,GSW_at_HOU_2016-04-21,114.890244
3,4,2016-04-24,3:30p,@,W,NaN,121.0,94.0,3.0,1.0,W 1,1,GSW,2016,HOU,GSW_at_HOU_2016-04-24,114.890244
4,5,2016-04-27,10:30p,NaN,W,NaN,114.0,81.0,4.0,1.0,W 2,1,GSW,2016,HOU,HOU_at_GSW_2016-04-27,114.890244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,18,2021-07-08,9:00p,NaN,W,NaN,118.0,108.0,14.0,4.0,W 3,1,PHO,2021,MIL,MIL_at_PHO_2021-07-08,115.277778
990,19,2021-07-11,8:00p,@,L,NaN,100.0,120.0,14.0,5.0,L 1,1,PHO,2021,MIL,PHO_at_MIL_2021-07-11,115.277778
991,20,2021-07-14,9:00p,@,L,NaN,103.0,109.0,14.0,6.0,L 2,1,PHO,2021,MIL,PHO_at_MIL_2021-07-14,115.277778
992,21,2021-07-17,9:00p,NaN,L,NaN,119.0,123.0,14.0,7.0,L 3,1,PHO,2021,MIL,MIL_at_PHO_2021-07-17,115.277778


In [5]:
dff = pd.DataFrame()

df
df_reg['ppg'] = 0


for team in df_reg['teamgame'].unique():
    dft = pd.DataFrame(df_reg[df_reg['teamgame'] == team].reset_index(drop=True))
    
    for szn in dft['season'].unique():
        dfts = pd.DataFrame(dft[dft['season'] == szn])
        
        dfts.sort_values(by=['Date'], inplace=True)
        
        dfts.reset_index(drop=True, inplace=True)
        
        for i in range(len(dfts)):
            if (dfts.loc[i, 'G'] > 1):
                dfts.loc[i, 'ppg'] = np.mean(dfts.loc[0:i-1,'Tm'])
            else:
                if (dfts.loc[i, 'season'] == 2016):
                    dfts.loc[i, 'ppg'] =  df_ppg[(df_ppg['teamgame'] == team) & (df_ppg['season'] == szn)]['ppg'].values[0]
                else:
                    dfts.loc[i, 'ppg'] =  df_ppg[(df_ppg['teamgame'] == team) & (df_ppg['season'] == szn-1)]['ppg'].values[0]
                
        dff = pd.concat([dff,dfts])

dff.reset_index(drop=True, inplace=True)

df_reg = dff.copy()

In [6]:
df = pd.concat([df_reg,df_pl]).sort_values(['Date']).reset_index(drop=True)



In [7]:
dff

,G,Date,Start (ET),home_away,result,ot,Tm,Opp,W,L,Streak,playoffs,teamgame,season,opponent,game_key,ppg
0,1,2015-10-27,8:00p,@,L,NaN,95.0,97.0,0.0,1.0,L 1,0,CLE,2016,CHI,CLE_at_CHI_2015-10-27,104.329268
1,2,2015-10-28,8:00p,@,W,NaN,106.0,76.0,1.0,1.0,W 1,0,CLE,2016,MEM,CLE_at_MEM_2015-10-28,95.000000
2,3,2015-10-30,7:00p,NaN,W,NaN,102.0,92.0,2.0,1.0,W 2,0,CLE,2016,MIA,MIA_at_CLE_2015-10-30,100.500000
3,4,2015-11-02,7:00p,@,W,NaN,107.0,100.0,3.0,1.0,W 3,0,CLE,2016,PHI,CLE_at_PHI_2015-11-02,101.000000
4,5,2015-11-04,8:00p,NaN,W,NaN,96.0,86.0,4.0,1.0,W 4,0,CLE,2016,NYK,NYK_at_CLE_2015-11-04,102.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15705,49,2022-01-28,8:00p,@,W,NaN,125.0,110.0,21.0,28.0,W 1,0,POR,2022,HOU,POR_at_HOU_2022-01-28,108.458333
15706,50,2022-01-30,3:30p,@,L,NaN,116.0,130.0,21.0,29.0,L 1,0,POR,2022,CHI,POR_at_CHI_2022-01-30,108.795918
15707,51,2022-01-31,8:00p,@,L,NaN,81.0,98.0,21.0,30.0,L 2,0,POR,2022,OKC,POR_at_OKC_2022-01-31,108.940000
15708,52,2022-02-02,10:30p,@,L,NaN,94.0,99.0,21.0,31.0,L 3,0,POR,2022,LAL,POR_at_LAL_2022-02-02,108.392157


In [8]:
away = df[df['home_away'] == '@'].reset_index(drop=True)
home = df[df['home_away'] != '@'].reset_index(drop=True)

away['wl_total'] = away['G']-1
home['wl_total'] = home['G']-1

away['win_pct'] = 0.0
home['win_pct'] = 0.0

for i in range(len(away)):
    if (away.loc[i,'result'] == 'L'):
        if (away.loc[i, 'wl_total'] == 0):
            away.loc[i, 'win_pct'] = 0.0
        elif (away.loc[i, 'wl_total'] > 0):
            away.loc[i, 'win_pct'] = away.loc[i, 'W']/(away.loc[i, 'G']-1)
    elif (away.loc[i,'result'] == 'W'):
        if (away.loc[i, 'wl_total'] == 0):
            away.loc[i, 'win_pct'] = 0.0
        else:
            away.loc[i, 'win_pct'] = (away.loc[i, 'W']-1)/(away.loc[i, 'G']-1)

away.rename(columns={'ppg':'away_ppg'}, inplace=True)
  
for i in range(len(home)):
    if (home.loc[i,'result'] == 'L'):
        if (home.loc[i, 'wl_total'] == 0):
            home.loc[i, 'win_pct'] = 0
        elif (home.loc[i, 'wl_total'] > 0):
            home.loc[i, 'win_pct'] = home.loc[i, 'W']/(home.loc[i, 'G']-1)
    elif (home.loc[i,'result'] == 'W'):
        if (home.loc[i, 'wl_total'] == 0):
            home.loc[i, 'win_pct'] = 0
        else:
            home.loc[i, 'win_pct'] = (home.loc[i, 'W']-1)/(home.loc[i, 'G']-1) 

            
home.rename(columns={'ppg':'home_ppg'}, inplace=True)

In [9]:
away[away['teamgame'] == 'ATL']

,G,Date,Start (ET),home_away,result,ot,Tm,Opp,W,L,Streak,playoffs,teamgame,season,opponent,game_key,away_ppg,wl_total,win_pct
17,2,2015-10-29,8:00p,@,W,NaN,112.0,101.0,1.0,1.0,W 1,0,ATL,2016,NYK,ATL_at_NYK_2015-10-29,94.000000,1,0.000000
44,4,2015-11-01,2:00p,@,W,NaN,94.0,92.0,3.0,1.0,W 3,0,ATL,2016,CHO,ATL_at_CHO_2015-11-01,101.000000,3,0.666667
58,5,2015-11-03,7:30p,@,W,NaN,98.0,92.0,4.0,1.0,W 4,0,ATL,2016,MIA,ATL_at_MIA_2015-11-03,99.250000,4,0.750000
82,7,2015-11-06,8:00p,@,W,NaN,121.0,115.0,6.0,1.0,W 6,0,ATL,2016,NOP,ATL_at_NOP_2015-11-06,99.333333,6,0.833333
135,11,2015-11-13,7:30p,@,L,NaN,93.0,106.0,8.0,3.0,L 1,0,ATL,2016,BOS,ATL_at_BOS_2015-11-13,104.400000,10,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8139,38,2022-01-07,10:00p,@,L,NaN,118.0,134.0,17.0,21.0,L 1,0,ATL,2022,LAL,ATL_at_LAL_2022-01-07,111.108108,37,0.459459
8151,39,2022-01-09,3:30p,@,L,NaN,93.0,106.0,17.0,22.0,L 2,0,ATL,2022,LAC,ATL_at_LAC_2022-01-09,111.289474,38,0.447368
8189,41,2022-01-14,8:00p,@,L,NaN,118.0,124.0,17.0,24.0,L 4,0,ATL,2022,MIA,ATL_at_MIA_2022-01-14,110.325000,40,0.425000
8252,46,2022-01-23,7:00p,@,W,NaN,113.0,91.0,21.0,25.0,W 4,0,ATL,2022,CHO,ATL_at_CHO_2022-01-23,111.200000,45,0.444444


In [10]:
away_pcts = pd.DataFrame(away[['game_key', 'win_pct', 'away_ppg']])

away_pcts.rename(columns = 
                {
                    'win_pct':'away_win_pct'
                },
                 
                inplace=True)

home.rename(columns = {'win_pct':'home_win_pct'}, inplace=True)

In [11]:
allg = home.merge(away_pcts, on='game_key', how='inner')

allg['pt_diff'] = allg['Tm'] - allg['Opp']

allg.insert(0, 'pk', 0)

for i in range(len(allg)):
    allg.loc[i, 'pk'] = i+1
    
allg.drop(columns=['home_away',  'ot'], inplace=True)

allg.rename(columns={
    'Tm':'score',
    'Opp':'opp_score'
}, inplace=True)

allg['mov'] = np.abs(allg['pt_diff'])


In [12]:
elo = pd.read_csv(r'nbaallelo.csv')

In [13]:
elo15 = pd.DataFrame(elo[elo['year_id']==2015]).reset_index(drop=True)

teamz = []
end_2015_elo = []

for team in elo15['team_id'].unique():
    teamz.append(team)
    
    d = pd.DataFrame(elo15[elo15['team_id'] == team])
    dd = d[d['gameorder'] == d['gameorder'].max()]['elo_n'].values[0]
    end_2015_elo.append(dd)

In [14]:
elo_15df = pd.DataFrame({
    'team':teamz,
    '2015_elo':end_2015_elo
})

elo_15df['team'] = elo_15df['team'].str.upper()
print(elo_15df)
elo_15df['2015_elo'] = (elo_15df['2015_elo']*.75)+(1505*.25)

   team   2015_elo
0   HOU  1654.6359
1   LAL  1283.2395
2   NOP  1526.5709
3   ORL  1311.8915
4   SAS  1720.9634
5   DAL  1556.5830
6   BOS  1524.5715
7   BRK  1457.7372
8   MIL  1443.2714
9   CHO  1400.7305
10  DEN  1422.3618
11  DET  1461.0784
12  IND  1504.8713
13  PHI  1276.1891
14  MEM  1608.3455
15  MIN  1263.9580
16  WAS  1546.1045
17  MIA  1455.8601
18  NYK  1255.6846
19  CHI  1591.0453
20  PHO  1466.8416
21  OKC  1583.2285
22  POR  1556.7389
23  GSW  1822.2881
24  SAC  1418.2802
25  TOR  1500.3939
26  ATL  1580.7914
27  UTA  1555.2854
28  CLE  1692.0859
29  LAC  1694.5431


In [15]:
def new_elo_calc(df, game_id):
    df1 = pd.DataFrame(df[df['pk'] == game_id].reset_index(drop=True))
    
    mov = df1.loc[0,'mov']
    eloh = elo_dict['%s_elo_ongoing' % df1.loc[0, 'teamgame']][len(elo_dict['%s_elo_ongoing' % df1.loc[0, 'teamgame']])-1]+100
    
    eloa = elo_dict['%s_elo_ongoing' % df1.loc[0, 'opponent']][len(elo_dict['%s_elo_ongoing' % df1.loc[0, 'opponent']])-1]
    
    if (df1.loc[0,'result'] == 'W'):
        elo_diff = eloh-eloa
        s_h = 1
        s_a = 0
    else:
        elo_diff = eloa-eloh
        s_h = 0
        s_a = 1
        
    k_num = (mov+3)**0.8     
    k_den = 7.5+0.006*(elo_diff)
    
    k = 20*(k_num/k_den)
    
    win_prob_h = 1/(1+(10**((eloa-eloh)/400)))
    win_prob_a = 1/(1+(10**((eloh-eloa)/400)))
    
    
    eloh_n = ((k*(s_h - win_prob_h))+eloh)-100
    eloa_n = k*(s_a - win_prob_a) + eloa
    
    elo_dict['%s_elo_ongoing' % df1.loc[0, 'teamgame']].append(eloh_n)
    elo_dict['%s_elo_ongoing' % df1.loc[0, 'opponent']].append(eloa_n)
    
    return eloh_n, eloa_n

In [16]:
team_elos = []
opp_elos = []
elo_dict = {}

for i in range(len(elo_15df)):
    elo_dict['%s_elo_ongoing' % elo_15df.loc[i,'team']] = []
    elo_dict['%s_elo_ongoing' % elo_15df.loc[i,'team']].append(elo_15df.loc[i, '2015_elo'])

    

allg16 = pd.DataFrame(allg[allg['season'] == 2016])
    
for i in range(len(allg16)):

    x, y = new_elo_calc(allg16, allg16.loc[i, 'pk'])
    
    team_elos.append(x)
    opp_elos.append(y)
    
for tm in elo_dict:
    elo_dict[tm].append((elo_dict[tm][-1] * 0.75) + (1505*0.25))

In [17]:
elo_dict

{'HOU_elo_ongoing': [1617.226925,
  1582.4373113930046,
  1569.669909609215,
  1553.3838526532506,
  1558.6859483635185,
  1560.5288276050198,
  1567.126060246444,
  1578.3748593519158,
  1559.3032582034605,
  1549.1310628376816,
  1531.8916416983986,
  1512.9309824706968,
  1517.4476034270401,
  1511.1306116188532,
  1497.820173442607,
  1487.1743559493002,
  1488.1553235067781,
  1494.8626613871925,
  1487.3080695647234,
  1492.0835697095713,
  1502.2973529399328,
  1506.129541627791,
  1499.464195383303,
  1509.8215841535343,
  1514.6137022809883,
  1506.8924711579334,
  1497.304668608894,
  1508.7436717198952,
  1518.864874410463,
  1524.5810081371408,
  1520.2038421222235,
  1529.7193148091794,
  1525.5684291034654,
  1515.697636890419,
  1512.6827977885723,
  1509.5591201268821,
  1516.1516596849385,
  1521.9959818107257,
  1527.189356617284,
  1546.6057162153688,
  1547.8243753420309,
  1536.0222810865614,
  1544.9529842193251,
  1540.8990291105902,
  1526.872329932507,
  1530.0

In [18]:
len(team_elos)

1316

In [19]:
ss

NameError: name 'ss' is not defined

In [ ]:
allg17 = pd.DataFrame(allg[allg['season'] == 2017]).reset_index(drop=True)
for i in range(len(allg17)):

    x, y = new_elo_calc(allg17, allg17.loc[i, 'pk'])

In [ ]:
elo_dict

In [ ]:
sd

In [ ]:
for szn in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    ally = pd.DataFrame(allg[allg['season'] == 2016])

     

    #     team_elos.append(x)
    #     opp_elos.append(y)

In [ ]:
elo_dict['HOU_elo_ongoing']

In [ ]:
allg15[(allg15['teamgame'] == 'HOU') | (allg15['opponent'] == 'HOU')]

In [ ]:
elo_dict

In [ ]:
team_elos = []
opp_elos = []
elo_dict = {}

for i in range(len(elo_15df)):
    elo_dict['%s_elo_ongoing' % elo_15df.loc[i,'team']] = []
    elo_dict['%s_elo_ongoing' % elo_15df.loc[i,'team']].append((elo_15df.loc[i, '2015_elo'])*0.75+(1505*.25))
    
for i in range(len(allg)):
#     s_h = 0
#     s_a = 0
    x, y = new_elo_calc(allg, allg.loc[i, 'pk']) 
    
    team_elos.append(x)
    opp_elos.append(y)

In [ ]:
allg['elo_i'] = team_elos
allg['opp_elo_i'] = opp_elos

In [ ]:
allg

In [ ]:
s

In [ ]:
wiz = elo15[elo15['team_id'] == 'was'].reset_index(drop=True)

wiz.loc[0]

### Random Dude's Method - it works!

##### https://www.ergosum.co/nate-silvers-nba-elo-algorithm/

In [ ]:
wiz

elo0 = wiz.loc[0, 'elo_i']
opp_elo0 = wiz.loc[0, 'opp_elo_i']+100
mov0 = np.abs(wiz.loc[0, 'pts'] - wiz.loc[0, 'opp_pts'])
elo_diff = opp_elo0 - elo0

k_num = (mov0+3)**0.8
k_den = 7.5+0.006*(elo_diff)

k = 20*(k_num/k_den)

e = 1/(1+(10**((opp_elo0-elo0)/400)))
elo1 = wiz.loc[1, 'elo_i']

fin = (k*-1*e)+elo0
fin


In [ ]:


elo0 = wiz.loc[1, 'elo_i']
opp_elo0 = wiz.loc[1, 'opp_elo_i']+100

opp_elo0
mov0 = np.abs(wiz.loc[1, 'pts'] - wiz.loc[1, 'opp_pts'])
elo_diff = elo0 - opp_elo0
elo_diff
k_num = (mov0+3)**0.8

k_num
k_den = 7.5+0.006*(elo_diff)

k = 20*(k_num/k_den)
k
e = 1/(1+(10**((opp_elo0-elo0)/400)))
# elo1 = wiz.loc[1, 'elo_i']

fin = (k*(1-e)+elo0)
fin

In [ ]:


elo0 = wiz.loc[2, 'elo_i']+100
opp_elo0 = wiz.loc[2, 'opp_elo_i']

opp_elo0
mov0 = np.abs(wiz.loc[2, 'pts'] - wiz.loc[2, 'opp_pts'])
elo_diff = elo0 - opp_elo0
elo_diff
k_num = (mov0+3)**0.8

k_num
k_den = 7.5+0.006*(elo_diff)

k = 20*(k_num/k_den)
k
e = 1/(1+(10**((opp_elo0-elo0)/400)))
# elo1 = wiz.loc[1, 'elo_i']

fin = (k*(1-e)+elo0)-100
fin

In [ ]:


elo_aw = wiz.loc[2, 'opp_elo_i']+100
opp_elo0 = wiz.loc[2, 'opp_elo_i']

opp_elo0
mov0 = np.abs(wiz.loc[2, 'pts'] - wiz.loc[2, 'opp_pts'])
elo_diff = elo0 - opp_elo0
elo_diff
k_num = (mov0+3)**0.8

k_num
k_den = 7.5+0.006*(elo_diff)

k = 20*(k_num/k_den)
k
e = 1/(1+(10**((opp_elo0-elo0)/400)))
# elo1 = wiz.loc[1, 'elo_i']

fin = (k*(1-e)+elo0)-100
fin

In [ ]:
wiz.loc[2]